In [33]:
import os
import git
from pathlib import Path

ROOT_DIR =  Path(git.Repo('.', search_parent_directories=True).working_tree_dir)
SAVE_FIGS = False

In [34]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
from reporting import *
plots_path = os.path.join(ROOT_DIR, "publication", "paper", "draft_plots")
main_df = main_df.copy()
RERUN=False
np.random.seed(0)

In [35]:
# (main_df['dataset'] + " (" + main_df['subset'].replace("full", "") + ")").str.replace(regex=r"\w+()", value="").value_counts()

In [36]:
main_df = main_df.fillna("None", inplace=False)
main_df['dataset'] = main_df['dataset'] + main_df['subset'].replace("full", "")

In [37]:
granularity = ['transform', 'dataset']
temp = main_df.copy()#[(main_df['dataset'] != 'standardTesting')] #  & (main_df['transform'] != 'learned')
temp['practically_pass'] = (temp['failure_category'] == 'practically_pass')
temp['actually_pass'] = (temp['failure_category'] == 'actually_pass')
summary_df = (temp.groupby(granularity)[['actually_pass','practically_pass','intersect_roi']].mean() * 100).round(1)
summary_df

actually_pass  practically_pass  \
transform dataset                                                   
fourier   agriVision                        2.3              93.2   
          pastis                           90.9               9.1   
          spaceNet                          0.0             100.0   
learned   agriVision                        0.0              78.3   
          cocoindoor                        0.0              56.5   
          cocooutdoor                       0.0              59.4   
          pastis                            0.0              85.7   
          segmentAnything                   0.0              73.4   
          spaceNet                          0.0              42.9   
          standardTesting                   0.0              24.1   
wavelet   agriVision                       13.9              86.1   
          cocoindoor                       25.0              25.0   
          cocooutdoor                      25.0              28.1   
          pastis                           57.1              42.9   
          segmentAnything                  25.0              60.2   
          spaceNet                          4.7              48.4   
          standardTesting                   9.5              38.1   
          syntheticMRI2Daxial               0.0              76.2   
          syntheticMRI2Dcoronal             9.5              71.4   
          syntheticMRI2Dsagittal            0.0              81.0   
          syntheticMRI3D                    7.1              61.9   

                                  intersect_roi  
transform dataset                                
fourier   agriVision                       70.5  
          pastis                          100.0  
          spaceNet                         50.0  
learned   agriVision                       51.7  
          cocoindoor                       45.2  
          cocooutdoor                      26.6  
          pastis                           77.1  
          segmentAnything                  51.6  
          spaceNet                         48.2  
          standardTesting                  70.4  
wavelet   agriVision                       75.0  
          cocoindoor                       37.5  
          cocooutdoor                      20.8  
          pastis                           96.4  
          segmentAnything                  38.9  
          spaceNet                         48.4  
          standardTesting                  19.0  
          syntheticMRI2Daxial              33.3  
          syntheticMRI2Dcoronal            57.1  
          syntheticMRI2Dsagittal           76.2  
          syntheticMRI3D                   66.7

In [38]:
granularity = ['dataset', 'transform']
temp = main_df.copy()[(main_df['dataset'] != 'standardTesting')]# & (main_df['transform'] != 'learned')]
temp['practically_pass'] = (temp['failure_category'] == 'practically_pass')
temp['actually_pass'] = (temp['failure_category'] == 'actually_pass')
temp['borderline'] = (temp['failure_category'] == 'borderline')
summary_df = (temp.groupby(granularity)[['actually_pass','practically_pass','intersect_roi']].mean() * 100).round(1).sort_values('transform')
summary_df.reset_index()#.drop('dataset', axis=1)

,dataset,transform,actually_pass,practically_pass,intersect_roi
0,agriVision,fourier,2.3,93.2,70.5
1,pastis,fourier,90.9,9.1,100.0
2,spaceNet,fourier,0.0,100.0,50.0
3,agriVision,learned,0.0,78.3,51.7
4,cocoindoor,learned,0.0,56.5,45.2
5,cocooutdoor,learned,0.0,59.4,26.6
6,pastis,learned,0.0,85.7,77.1
7,spaceNet,learned,0.0,42.9,48.2
8,segmentAnything,learned,0.0,73.4,51.6
9,syntheticMRI2Dcoronal,wavelet,9.5,71.4,57.1


In [39]:
save_path = Path(os.path.join(ROOT_DIR, 'publication', 'poster', 'CSVs', 'summary_table.csv'))
summary_df.to_csv(save_path) #.drop('learned')

In [40]:
summary_df = summary_df.reset_index()
summary_df

,dataset,transform,actually_pass,practically_pass,intersect_roi
0,agriVision,fourier,2.3,93.2,70.5
1,pastis,fourier,90.9,9.1,100.0
2,spaceNet,fourier,0.0,100.0,50.0
3,agriVision,learned,0.0,78.3,51.7
4,cocoindoor,learned,0.0,56.5,45.2
5,cocooutdoor,learned,0.0,59.4,26.6
6,pastis,learned,0.0,85.7,77.1
7,spaceNet,learned,0.0,42.9,48.2
8,segmentAnything,learned,0.0,73.4,51.6
9,syntheticMRI2Dcoronal,wavelet,9.5,71.4,57.1


In [41]:
(temp[['actually_pass','practically_pass','intersect_roi', 'borderline']].mean(numeric_only=True)*100).round(1)

actually_pass       15.7
practically_pass    57.2
intersect_roi       52.4
borderline           4.9
dtype: float64

In [42]:
(temp.groupby(['transform'])[['kstest_stat_best', 'actually_pass','practically_pass','intersect_roi', 'total_samples']].median(numeric_only=True)*1)#.round(1)

,kstest_stat_best,actually_pass,practically_pass,intersect_roi,total_samples
transform,,,,,
fourier,0.003361,0.0,1.0,1.0,552780.0
learned,0.007980,0.0,1.0,0.0,480000000.0
wavelet,0.006903,0.0,1.0,1.0,626176.0


In [43]:
(temp.groupby(['dataset', 'transform'])[['kstest_stat_best', 'actually_pass','practically_pass','intersect_roi', 'total_samples']].median(numeric_only=True)*1)#.round(1)

kstest_stat_best  actually_pass  \
dataset                transform                                    
agriVision             fourier            0.004708            0.0   
                       learned            0.005417            0.0   
                       wavelet            0.005020            0.0   
cocoindoor             learned            0.009850            0.0   
                       wavelet            0.013062            0.0   
cocooutdoor            learned            0.008832            0.0   
                       wavelet            0.011024            0.0   
pastis                 fourier            0.001526            1.0   
                       learned            0.006469            0.0   
                       wavelet            0.004502            1.0   
segmentAnything        learned            0.004959            0.0   
                       wavelet            0.004032            0.0   
spaceNet               fourier            0.005150            0.0   
                       learned            0.012170            0.0   
                       wavelet            0.011693            0.0   
syntheticMRI2Daxial    wavelet            0.007145            0.0   
syntheticMRI2Dcoronal  wavelet            0.004148            0.0   
syntheticMRI2Dsagittal wavelet            0.005115            0.0   
syntheticMRI3D         wavelet            0.004573            0.0   

                                  practically_pass  intersect_roi  \
dataset                transform                                    
agriVision             fourier                 1.0            1.0   
                       learned                 1.0            1.0   
                       wavelet                 1.0            1.0   
cocoindoor             learned                 1.0            0.0   
                       wavelet                 0.0            0.0   
cocooutdoor            learned                 1.0            0.0   
                       wavelet                 0.0            0.0   
pastis                 fourier                 0.0            1.0   
                       learned                 1.0            1.0   
                       wavelet                 0.0            1.0   
segmentAnything        learned                 1.0            1.0   
                       wavelet                 1.0            0.0   
spaceNet               fourier                 1.0            0.5   
                       learned                 0.0            0.0   
                       wavelet                 0.0            0.0   
syntheticMRI2Daxial    wavelet                 1.0            0.0   
syntheticMRI2Dcoronal  wavelet                 1.0            1.0   
syntheticMRI2Dsagittal wavelet                 1.0            1.0   
syntheticMRI3D         wavelet                 1.0            1.0   

                                  total_samples  
dataset                transform                 
agriVision             fourier        2574000.0  
                       learned      786432000.0  
                       wavelet        1728000.0  
cocoindoor             learned      196608000.0  
                       wavelet         256640.0  
cocooutdoor            learned      196608000.0  
                       wavelet         391360.0  
pastis                 fourier         257580.0  
                       learned         491520.0  
                       wavelet         152640.0  
segmentAnything        learned      786432000.0  
                       wavelet        1810432.0  
spaceNet               fourier        1816134.0  
                       learned      480000000.0  
                       wavelet        1637581.5  
syntheticMRI2Daxial    wavelet        4185552.0  
syntheticMRI2Dcoronal  wavelet        3670424.0  
syntheticMRI2Dsagittal wavelet        2418270.0  
syntheticMRI3D         wavelet        1588712.0